# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
!git branch -a

/content
Cloning into 'project'...
remote: Enumerating objects: 594, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 594 (delta 80), reused 80 (delta 46), pack-reused 430 (from 1)
Receiving objects: 100% (594/594), 1.42 MiB | 9.25 MiB/s, done.
Resolving deltas: 100% (270/270), done.
/content/project
* main
  remotes/origin/16-steps-for-phase-1
  remotes/origin/HEAD -> origin/main
  remotes/origin/feat/create-gradio-ui
  remotes/origin/feat/preprocess-the-data-into-a-normalized-format
  remotes/origin/fix/imports-and-file-paths
  remotes/origin/main
  remotes/origin/prompting/Deepseek
  remotes/origin/prompting/Gemini
  remotes/origin/refactor/docs-and-APIs
  remotes/origin/refactor/py-and-output-documents
  remotes/origin/scraping/body
  remotes/origin/scraping/inline_feedback
  remotes/origin/scraping/rubric_table
  remotes/origin/wip/prompting


In [2]:
# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout scraping/rubric_table

Branch 'scraping/rubric_table' set up to track remote branch 'scraping/rubric_table' from 'origin'.
Switched to a new branch 'scraping/rubric_table'


**--Install required libraries and set paths**

In [3]:
%pip install -r requirements.txt
!pip install python-docx

# import data path for loading files
import os

# Base data directory
data_base = '/content/project/data'

# Paths to specific subfolders
raw_data_path = os.path.join(data_base, 'raw')
processed_data_path = os.path.join(data_base, 'processed')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 3.8 MB/s eta 0:00:00


**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

## Rubric scraper

In [9]:
# Add your src folder to the Python path
import sys
sys.path.append('/content/project/')

from tropos.preprocess_docx.rubric import Rubric, parse_rubric

from docx import Document

# loop through tables and cells
doc_path = "/content/project/data/raw/Student 1/Student 1 Part 1.docx"
rubric = parse_rubric(doc_path)


In [12]:
import json
rubric = parse_rubric("/content/project/data/raw/Student 1/Student 1 Part 1.docx")

# Print criteria with pretty formatting
print(json.dumps(rubric.get_criteria(), indent=2))

[
  {
    "portion": "Introduction",
    "criteria": {
      "text": "Engaging\nShows importance of question\nIndicates major parts and content"
    },
    "feedback": ""
  },
  {
    "portion": "Background",
    "criteria": {
      "text": "Uses credible, neutral sources\nEffectively and sufficiently explains essential information\nWritten objectively in own words\nWell organized and focused paragraphs with transitions\nMinimal error"
    },
    "feedback": "I like the history parts.\n\nWhat\u2019s missing is more about what the mandate looked like: how it was \u201cenforced\u201d and a little more about wearing masks properly."
  },
  {
    "portion": "Analysis",
    "criteria": {
      "text": "Smooth transition from background\nSummarizes accurately and in own words two opposing answers and one objective answer to the question\nRecognizes and shows bias\nReasonable observations about how all sources make their arguments impactful\nWell-organized and focused paragraphs with transiti